# ***Kaggle:*** Chess Evaluations

| Author's Name         | NIU     |
| --------------------- | ------- |
| Albert Capdevila      | 1587933 |

<small>*This Jupyter Notebook is submitted as an assignment for the Machine Learning subject in the Computer Engineering degree at the Universitat Autònoma de Barcelona (UAB).*<small>

## 1. Problem definition

In computer chess, an evaluation function estimates the strength of a position by assigning a numerical score that indicates which side (White or Black) has the advantage and by how much.

This score is based on the values of chess pieces, which are traditionally:
| Piece  | Pawn | Knight | Bishop | Rook | Queen |
|--------|------|--------|--------|------|-------|
| Symbol | ![Pawn](https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Chess_plt45.svg/45px-Chess_plt45.svg.png) | ![Knight](https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Chess_nlt45.svg/45px-Chess_nlt45.svg.png) | ![Bishop](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Chess_blt45.svg/45px-Chess_blt45.svg.png) | ![Rook](https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Chess_rlt45.svg/45px-Chess_rlt45.svg.png) | ![Queen](https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Chess_qlt45.svg/45px-Chess_qlt45.svg.png) |
| Value  | 1    | 3      | 3      | 5    | 9     |

<small>*The value of the king is undefined as it cannot be captured, let alone traded, during the course of the game. Chess engines usually assign the king an arbitrary large value such as 200 points [¹](#references).*</small>

These values show that, for example, capturing a bishop is equivalent to capturing three pawns. Therefore, all of them can be expressed in *'pawn units'*.

It’s also important to note that evaluations are opposite, with positive values indicating an advantage for White and negative values for Black. For example, a negative evaluation of $-3$ pawns may indicate that Black has an advantage equivalent to three pawns, a bishop, or a knight.

Usually, these evaluations are calculated by complex algorithms, such as *Stockfish*, which are based on **minimax** principles, calculating, pruning, and predicting the best moves across multiple levels of depth. These algorithms can be slow and computationally intensive, as they analyze many possible future moves in a position. This notebook will explore the use of regression models to achieve similar results more quickly.

***In this notebook, we aim to estimate the evaluation score of a chess position through feature extraction and simple regression techniques.***



## 2. Data collection

For this machine learning problem, we will use the **Chess Evaluations** dataset, available [here](https://www.kaggle.com/datasets/ronakbadhe/chess-evaluations/data) on Kaggle. The dataset contains two columns: 

| Feature             | Description                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| **Position (FEN)**  | This is a string representing the chess position in Forsyth-Edwards Notation (FEN). |
| **Evaluation**      | This is the evaluation of the position in ***centipawns***. `#` means that there is forced checkmate. |

The evaluations are in centi-pawns and are generated from Stockfish 11 at depth 22.

Let’s download and import the dataset:

In [49]:

from kaggle.api.kaggle_api_extended import KaggleApi
from zipfile import ZipFile
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import os 

api = KaggleApi()
api.authenticate()

dataser_owner = 'ronakbadhe'
dataset_name= 'chess-evaluations'
dataset = f'{dataser_owner}/{dataset_name}'
target = 'Evaluation'

if not os.path.isdir('data'):
  print("The 'data' directory was not found")
  print("Downloading the dataset from Kaggle...")
  os.mkdir('data')
  api.dataset_download_files(dataset = dataset,path = 'data')
  zf = ZipFile(f'data/{dataset_name}.zip')
  zf.extractall(f'data')
  zf.close()
  os.remove(f'data/{dataset_name}.zip')
  print("Dataset downloaded successfully ✓")
else:
  print("The 'data' directory was found successfully ✓")

The 'data' directory was found successfully ✓


In [50]:
csvFile = "chessData.csv"
print(f"Reading the file '{csvFile}'...")

df = pd.read_csv(f'data/{csvFile}')

print(f"File '{csvFile}' read successfully ✓")

Reading the file 'chessData.csv'...
File 'chessData.csv' read successfully ✓


In [51]:
# Reducing the dataset size to make it faster to work with
from sklearn.model_selection import train_test_split

_, df = train_test_split(df, test_size=0.001, random_state=42)

In [52]:
df.head()

,FEN,Evaluation
11784899,r3k2r/1b2bppp/p1n1pn2/1p2N1B1/8/2NB1P2/PPP3PP/...,-35
8860705,r3k2r/1pp2p2/1bnp1q2/p2Bp1p1/PP2P1bP/1QPP1N2/3...,-201
7376649,6k1/2p3pp/8/8/P3R3/3r1PP1/5K1P/8 b - - 0 30,+121
2224083,1r6/2rnppkp/1q1p2n1/2pP2p1/2b5/1P4PP/P1QNPPB1/...,+280
4108821,5k2/3R4/1p1p4/3n2pp/8/8/PP6/6K1 b - - 1 35,+328


## 3. Data preprocessing

### 3.1 Missing Data

In [53]:
df.isna().sum()

FEN           0
Evaluation    0
dtype: int64

Since there are no missing values, there is no need to handle NaN values.

### 3.2 Data Conversion

In [54]:
df.dtypes

FEN           object
Evaluation    object
dtype: object

The evaluation feature is a string that starts with either a "+" or "-" symbol. To use it in the model, it will need to be converted into a numeric value.

It may also include the character `#`, but since this is not relevant to the current problem, we will ignore it.

In [55]:
tqdm.pandas(desc="Converting 'Evaluation' to numeric values")
df['Evaluation'] = df['Evaluation'].progress_apply(lambda x: int(x) if '#' not in x else int(x.replace('#', '')))

Converting 'Evaluation' to numeric values: 100%|██████████| 12959/12959 [00:00<00:00, 341320.88it/s]


In [56]:
df.dtypes

FEN           object
Evaluation     int64
dtype: object

### 3.3 Data Normalization

Apart from the target feature (Evaluation), the dataset contains only one feature: FEN, which cannot be normalized. Therefore, normalization is not applicable in this case.

### 3.4 Feature extraction

In this section, we will focus on feature extraction, which is the most important step in building our model to predict the evaluation of a chess position. At the moment, we only have the FEN notation of the chess position, and our goal is to extract useful features from it.

#### 3.4.1 Information in the FEN Notation

Forsyth–Edwards Notation (FEN) is a standard notation for describing a particular board position of a chess game.

A typical FEN string looks like this: `rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1`

The FEN string above represents the following board position:

<img src="img/example_board.svg" alt="Example Board" width="300"/>

A FEN string has six fields, each separated by a space:

1. **Piece placement data:**

    It is the arrangement of pieces on the board, row by row, from top to bottom. Each piece is represented by a letter (uppercase for White and lowercase for Black). Empty squares are indicated by numbers.

2. **Active color:**

    Indicates which side is to move (`w` for White, `b` for Black).

3. **Castling availability:**

    If no castling is possible, this field is `-`. Otherwise, it shows `K`, `Q`, `k`, or `q` for available castling options.

4. **En passant target square:**

    Indicates the square where an en passant capture is possible, or `-` if not.

5. **Halfmove clock:**

    Counts the number of halfmoves since the last pawn advance or capture, used for the 50-move rule. 

6. **Fullmove number:**

    Tracks the total number of moves in the game, starting at 1 and incrementing after each Black move.

Only using this information, we can start extracting some simple new features:

In [57]:
def count_white_pieces(fen):
    return len(re.findall(r'[KQRBNP]', fen))

def count_black_pieces(fen):
    return len(re.findall(r'[kqrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_pieces_count'")
df['White_pieces_count'] = df['FEN'].progress_apply(count_white_pieces)
tqdm.pandas(desc="Adding column 'Black_pieces_count'")
df['Black_pieces_count'] = df['FEN'].progress_apply(count_black_pieces)

Adding column 'Black_pieces_count': 100%|██████████| 12959/12959 [00:00<00:00, 154277.55it/s]


In [58]:
piece_values = {
    'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9,
    'p': -1, 'n': -3, 'b': -3, 'r': -5, 'q': -9
}

# The king's value is ignored for now.

def get_white_material(fen):
    return sum(piece_values[piece] for piece in re.findall(r'[QRBNP]', fen))

def get_black_material(fen):
    return -sum(piece_values[piece] for piece in re.findall(r'[qrbnp]', fen))

tqdm.pandas(desc="Adding column 'White_material'")
df['White_material'] = df['FEN'].progress_apply(get_white_material)

tqdm.pandas(desc="Adding column 'Black_material'")
df['Black_material'] = df['FEN'].progress_apply(get_black_material)

df["Material_difference"] = df['White_material'] - df['Black_material']
print("Added column 'Material_difference' ✓")

Adding column 'Black_material': 100%|██████████| 12959/12959 [00:00<00:00, 122254.50it/s]

Added column 'Material_difference' ✓


In [59]:
def get_turn(fen):
    return 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'Turn'")
df['Turn'] = df['FEN'].progress_apply(get_turn)

Adding column 'Turn': 100%|██████████| 12959/12959 [00:00<00:00, 518407.46it/s]


In [60]:
def get_castling(fen, char):
    return int(char in fen.split(' ')[2])

tqdm.pandas(desc="Adding column 'White_king_castling'")
df['White_king_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'K'))

tqdm.pandas(desc="Adding column 'White_queen_castling'")
df['White_queen_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'Q'))

tqdm.pandas(desc="Adding column 'Black_king_castling'")
df['Black_king_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'k'))

tqdm.pandas(desc="Adding column 'Black_queen_castling'")
df['Black_queen_castling'] = df['FEN'].progress_apply(lambda fen: get_castling(fen, 'q'))

df['Castling_difference'] = df['White_king_castling'] + df['White_queen_castling'] - df['Black_king_castling'] - df['Black_queen_castling']
print("Added column 'Castling_difference' ✓")


Adding column 'Black_queen_castling': 100%|██████████| 12959/12959 [00:00<00:00, 446839.35it/s]

Added column 'Castling_difference' ✓


In [61]:
def get_en_passant(fen):
    # This feature is 1 if en passant is available for White, -1 for Black, and 0 if none
    return int(fen.split(' ')[3] != '-') * 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'En_passant'")
df['En_passant'] = df['FEN'].progress_apply(get_en_passant)

Adding column 'En_passant': 100%|██████████| 12959/12959 [00:00<00:00, 404992.07it/s]


In [62]:
def get_rule50(fen):
    return int(fen.split(' ')[4]) * 1 if fen.split(' ')[1] == 'w' else -1

tqdm.pandas(desc="Adding column 'Rule_50'")
df['Rule_50'] = df['FEN'].progress_apply(get_rule50)

Adding column 'Rule_50': 100%|██████████| 12959/12959 [00:00<00:00, 350252.83it/s]


In [63]:
def get_n_moves(fen):
    return int(fen.split(' ')[5]) + int(fen.split(' ')[1] == 'b')

tqdm.pandas(desc="Adding column 'N_moves'")
df['N_moves'] = df['FEN'].progress_apply(get_n_moves)

Adding column 'N_moves': 100%|██████████| 12959/12959 [00:00<00:00, 350255.09it/s]


Let's see the correlations between the new features and the target variable:

In [64]:
correlations = df.corr(numeric_only=True)['Evaluation']
sorted_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index)
print("Correlations with 'Evaluation' (sorted by absolute value):")
print(sorted_correlations)

Correlations with 'Evaluation' (sorted by absolute value):
Evaluation              1.000000
Material_difference     0.195666
White_pieces_count      0.052898
White_material          0.036804
Black_pieces_count     -0.031224
En_passant              0.028921
Turn                    0.028843
Rule_50                 0.021402
Black_material         -0.017215
N_moves                -0.007546
Black_king_castling     0.001712
Castling_difference    -0.001312
White_king_castling    -0.000689
White_queen_castling   -0.000240
Black_queen_castling   -0.000061
Name: Evaluation, dtype: float64


There are no significant correlations.

Currently, we only extract information from the FEN string, not from the position itself (for instance, two positions with the same pieces, turn, en passant, and number of moves would appear identical to the model).

We need to extract information directly from the game. To achieve this, we'll use the chess library to obtain a board representation and apply some of its functions to build new features.

In [65]:
%pip install chess
import chess

Note: you may need to restart the kernel to use updated packages.


In [66]:
def get_board(fen):
    return chess.Board(fen)

tqdm.pandas(desc="Adding column 'Board'")
df['Board'] = df['FEN'].progress_apply(get_board)

Adding column 'Board': 100%|██████████| 12959/12959 [00:01<00:00, 9514.40it/s]


Here are a few features I think could have a strong correlation with the evaluation:

* **Attack count:**
    It is the number of squares attacked by each individual piece of a player. If a square is attacked by more than one piece, it is counted multiple times.

In [67]:
def get_attacks_count(board, color):
    return sum(len(board.attackers(color, square)) for square in chess.SQUARES)

tqdm.pandas(desc="Adding column 'White_attacks'")
df['White_attacks'] = df['Board'].progress_apply(lambda board: get_attacks_count(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_attacks'")
df['Black_attacks'] = df['Board'].progress_apply(lambda board: get_attacks_count(board, chess.BLACK))

df["Attacks_difference"] = df['White_attacks'] - df['Black_attacks']
print("Added column 'Attacks_difference' ✓")

Adding column 'Black_attacks': 100%|██████████| 12959/12959 [00:02<00:00, 5026.77it/s]

Added column 'Attacks_difference' ✓


* **Legal moves:**
    It is the total number of legal moves available for a player in the current position.

In [68]:
def get_legal_moves(board, color):
    orig_turn = board.turn 
    board.turn = color
    legal_moves = len(list(board.legal_moves))
    board.turn = orig_turn
    return legal_moves

tqdm.pandas(desc="Adding column 'White_legal_moves'")
df['White_legal_moves'] = df['Board'].progress_apply(lambda board: get_legal_moves(board, chess.WHITE))

tqdm.pandas(desc="Adding column 'Black_legal_moves'")
df['Black_legal_moves'] = df['Board'].progress_apply(lambda board: get_legal_moves(board, chess.BLACK))

df["Legal_moves_difference"] = df['White_legal_moves'] - df['Black_legal_moves']
print("Added column 'Legal_moves_difference' ✓")

Adding column 'Black_legal_moves': 100%|██████████| 12959/12959 [00:01<00:00, 11028.96it/s]

Added column 'Legal_moves_difference' ✓


* **Attacked pieces**:
    It is the total number of opponent attacks on squares occupied by a player's pieces.

In [69]:
def get_attacked_pieces(board, color):
    attacked_pieces = 0
    opponent_color = chess.WHITE if color == chess.BLACK else chess.BLACK 
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece and piece.color == color:
            attacked_pieces += len(board.attackers(opponent_color, square))

    return attacked_pieces

tqdm.pandas(desc="Adding column 'White_attacked'")
df['White_attacked'] = df['Board'].progress_apply(lambda board: get_attacked_pieces(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_attacked'")
df['Black_attacked'] = df['Board'].progress_apply(lambda board: get_attacked_pieces(board, chess.BLACK))

df["Attacked_difference"] = df['Black_attacked'] - df['White_attacked']
print("Added column 'Attacked_difference' ✓")

Adding column 'White_attacked':   0%|          | 0/12959 [00:00<?, ?it/s]

Adding column 'Black_attacked': 100%|██████████| 12959/12959 [00:00<00:00, 13156.40it/s]

Added column 'Attacked_difference' ✓


- **Check:** Verifies if a player's king is under attack

In [70]:
def get_check(board):
    return int(board.is_check()) * 1 if board.turn else -1

tqdm.pandas(desc="Adding column 'Check'")
df['Check'] = df['Board'].progress_apply(get_check)

Adding column 'Check': 100%|██████████| 12959/12959 [00:00<00:00, 332314.26it/s]


<small>*"In chess, doubled pawns are two pawns of the same color residing on the same file. Pawns can become doubled only when one pawn captures onto a file on which another friendly pawn resides. [...] In most cases, doubled pawns are considered a weakness due to their inability to defend each other." [²](#references)*</small>


* **Doubled pawns**: It is the count of how many extra pawns are stacked in each file for a player

In [71]:
def count_doubled_pawns(board, color):
    doubled_pawns = 0
    for file in chess.FILE_NAMES:
        column_squares = [chess.square(chess.FILE_NAMES.index(file), rank) for rank in range(8)]
        pawns_in_file = sum(1 for square in column_squares if board.piece_at(square) == chess.Piece(chess.PAWN, color))
        if pawns_in_file > 1:
            doubled_pawns += pawns_in_file - 1
    return doubled_pawns

tqdm.pandas(desc="Adding column 'White_doubled_pawns'")
df['White_doubled_pawns'] = df['Board'].progress_apply(lambda board: count_doubled_pawns(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_doubled_pawns'")
df['Black_doubled_pawns'] = df['Board'].progress_apply(lambda board: count_doubled_pawns(board, chess.BLACK))

df["Doubled_pawns_difference"] = df['White_doubled_pawns'] - df['Black_doubled_pawns']
print("Added column 'Doubled_pawns_difference' ✓")


Adding column 'Black_doubled_pawns': 100%|██████████| 12959/12959 [00:01<00:00, 7766.89it/s]

Added column 'Doubled_pawns_difference' ✓


<small>*"In chess, a fork is a tactic in which a piece attacks multiple enemy pieces simultaneously. The attacker usually aims to capture one of the forked pieces" [³](#references)*</small>

- **Forks:** Counts the number of forks for each player.

In [72]:
def get_forks(board, color):
    forks = 0
    for square, piece in board.piece_map().items():
        if piece.color == color:
            attacked_squares = board.attacks(square)
            if len([target for target in attacked_squares if board.piece_at(target) and board.piece_at(target).color != color]) > 1:
                forks += 1

    return forks

tqdm.pandas(desc="Adding column 'White_forks'")
df['White_forks'] = df['Board'].progress_apply(lambda board: get_forks(board, chess.WHITE))
tqdm.pandas(desc="Adding column 'Black_forks'")
df['Black_forks'] = df['Board'].progress_apply(lambda board: get_forks(board, chess.BLACK))

df["Forks_difference"] = df['White_forks'] - df['Black_forks']
print("Added column 'Forks_difference' ✓")

Adding column 'Black_forks': 100%|██████████| 12959/12959 [00:01<00:00, 8129.86it/s]

Added column 'Forks_difference' ✓


Let's see the current strongest correlations: 

In [73]:
correlations = df.corr(numeric_only=True)['Evaluation']
sorted_correlations = correlations.reindex(correlations.abs().sort_values(ascending=False).index)
print("Correlations with 'Evaluation' (sorted by absolute value):")
print(sorted_correlations)

Correlations with 'Evaluation' (sorted by absolute value):
Evaluation                  1.000000
Attacks_difference          0.232247
Material_difference         0.195666
Legal_moves_difference      0.178131
White_legal_moves           0.096346
Black_legal_moves          -0.082910
White_attacks               0.069455
White_pieces_count          0.052898
Black_attacks              -0.051794
White_material              0.036804
Black_pieces_count         -0.031224
White_forks                 0.030342
En_passant                  0.028921
Turn                        0.028843
Attacked_difference         0.026411
Black_attacked              0.025376
Forks_difference            0.024751
Rule_50                     0.021402
Doubled_pawns_difference    0.021246
Black_doubled_pawns        -0.017758
Black_material             -0.017215
White_doubled_pawns         0.011842
Check                       0.008928
N_moves                    -0.007546
White_attacked             -0.005979
Black_forks     

Using the chess library, we can extract much more information, but we must keep it simple.  

So far, we have focused on attributes that indicate whether a position might be better, but we have ignored the fact that the goal is to infer a specific value: the evaluation provided by Stockfish 11 at 22 levels of depth (considering 22 turns ahead).  

Therefore, we need to identify the features that Stockfish use to generate their evaluations.

Stockfish's evaluation function is actually quite complex and uses lots of values to determine its result. We'll focus on the main ones and simplify where needed:

- Middle game evaluation

    - **Piece Value (MG)**: Material is weighted differently during the middlegame.

    - **Psqt (MG)**: Piece-square table bonuses, where certain positions provide an advantage for specific pieces in the middlegame.

    - Pawns:
        - Doubled: *(Simplified: Already considered in 'Double_pawns')*

    - Pieces:
        - **Minor behind pawn**: Knight or bishop when behind a pawn
        - **Bishop pawns**: Number of pawns on the same color square as the bishop *(Simplified)*
        - **Bishop xray pawns**: Advantatge for every opponent pawn in the diagonal of a bishop
        - **Rook on queen file**: Simple bonus for a rook that is on the same file as any queen
        - **Rook on file**: Rook when on an open file *(Simplified)*
        - **Long diagonal bishop**: Bonus for bishop on a long diagonal which can "see" both center squares.

    - Mobility: *(Simplified: Already considered in 'Legal_moves')*

    - Threats
        - **Hanging**: Weak enemies not defended by opponent *(Simplified)*
        - **Rook threat**: Threat type for attacked by rook pieces *(Simplified)*

    - **Passed**: Bonuses for passed pawns *(Simplified)*

    - **King**: Bonuses for player's pieces around the king *(Simplified)* 

- End game evaluation

    - **Piece Value (EG)**: Material is weighted differently during the endgame.

    - **Psqt (EG)**: Piece-square table bonuses, where certain positions provide an advantage for specific pieces in the endgame.

    It also takes all the above into account, with different weights *(Will be ignored for simplicity)*

- **Phase**: Eval based on the amount of non-pawn material on the board *(Simplified)*

- Rule50: *(Already considered in 'Rule_50')*

- Scale factor: The scale factors are used to scale the endgame evaluation score down *(It will also be ignored)*

- Tempo: *(Already considered in 'Turn')*

<small>*All the bold titles represent new attributes that will be added.*</small>

These few added columns will be representations of the board, which will be useful for calculating attributes faster.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
clf = RandomForestRegressor()

X, y = df.drop(columns=[target, 'FEN', 'Board']), df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

r2_score(y_test, y_pred)

ValueError: setting an array element with a sequence.

## References

1. [Chess Piece Relative Value - Wikipedia](https://en.wikipedia.org/wiki/Chess_piece_relative_value)
2. [Doubled pawns - Wikipedia](https://en.wikipedia.org/wiki/Doubled_pawn)
3. [Fork (chess) - Wikipedia](https://en.wikipedia.org/wiki/Fork_(chess))